## Visits Data

In [1]:
import pandas as pd
import numpy as np
from statsmodels.robust import mad
import math
import datetime
import json
import warnings
import re
warnings.filterwarnings('ignore')

# Visits Data - Product (Target Variable) - US All Product (Web-App) Visits
def visits_product(file_path,file_path1):
    df_v1 = pd.read_excel(file_path+"US All Product (Web-App) Visits 2021-10.17.23 - REVISED 1.4.24.xlsx")
    df_v2 = pd.read_excel(file_path+"US All Product (Web-App) Visits 10.1.23-12.31.23.xlsx")
    df_v3 = pd.read_excel(file_path1+"US All Product (Web-App) Visits 1.1.24-5.31.24 (3).xlsx")
    df_v1['Date'] = pd.to_datetime(df_v1['Date'])
    df_v2['Date'] = pd.to_datetime(df_v2['Date'])
    df_v3['Date'] = pd.to_datetime(df_v3['Date'])
    df_v1 = df_v1[df_v1['Date']<'2023-10-01']
    df_ = pd.concat([df_v1, df_v2])
    df_ = pd.concat([df_, df_v3])
    df_['% of Goal']=df_['% of Goal'].replace('-', pd.NA)
    df_['US Weather Impact Rating']=df_['US Weather Impact Rating'].replace('-',pd.NA)
    return df_

In [2]:
# Call the function
file_path = "C:\\Users\\DeepakArun\\Desktop\\TWC\\Data\\"
file_path1 = "C:\\Users\\DeepakArun\\Downloads\\new data\\"  # Replace with your actual file path
df = visits_product(file_path,file_path1)

In [3]:

df.to_clipboard()

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4988 entries, 0 to 607
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Region                    4988 non-null   object        
 1   Platform                  4988 non-null   object        
 2   Brand                     4988 non-null   object        
 3   Product                   4988 non-null   object        
 4   Date                      4988 non-null   datetime64[ns]
 5   Metric                    4988 non-null   object        
 6   Value                     4988 non-null   float64       
 7   Goal                      4866 non-null   float64       
 8   % of Goal                 1460 non-null   object        
 9   US Weather Impact Rating  4860 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 428.7+ KB


In [83]:
def visits_product_aggregate(file_path):
    df_ = visits_product(file_path,file_path1)
    df_grp = df_.groupby('Date').agg({'Value':'sum', 'Goal':'sum', '% of Goal':'mean'}).reset_index()
    df_grp = pd.merge(df_grp, df_[['Date','US Weather Impact Rating']].drop_duplicates().sort_values('Date').reset_index(drop=True), on='Date')
    df_grp['% of Goal'] = df_grp['Value']/df_grp['Goal']
    df_grp['% of Goal'] = df_grp['% of Goal'].replace([np.inf, -np.inf], 0)
    return df_grp

In [88]:
# Pivot the DataFrame
df = pd.read_excel("C:\\Users\\DeepakArun\\Desktop\\Latest visits an weather data.xlsx")
pivot_df = df.pivot(index='Date', columns='Product', values='Value').reset_index()

# Display the result
pivot_df

Product,Date,TWC Mobile Web,TWC Universal Android 4G+,TWC Universal iOS 4G+,TWC Web
0,2021-01-01,12464048.0,10826157.0,29938503.0,3027135.0
1,2021-01-02,9273872.0,8603391.0,23064532.0,2657871.0
2,2021-01-03,9370251.0,8328896.0,22867260.0,2688370.0
3,2021-01-04,7248857.0,6941222.0,17900246.0,3212981.0
4,2021-01-05,6968150.0,7109226.0,17841410.0,3079061.0
...,...,...,...,...,...
1242,2024-05-27,10011685.0,14010423.0,43922067.0,2761178.0
1243,2024-05-28,8504932.0,11717852.0,37447803.0,3611440.0
1244,2024-05-29,6746428.0,9604643.0,30353054.0,3024320.0
1245,2024-05-30,6569723.0,9024250.0,30144592.0,2811162.0


In [89]:
pivot_df.to_clipboard()

In [84]:
# Call the function
file_path = "C:\\Users\\DeepakArun\\Desktop\\TWC\\Data\\"
file_path1 = "C:\\Users\\DeepakArun\\Downloads\\new data\\"  # Replace with your actual file path
df_grp = visits_product_aggregate(file_path)

In [86]:
df_grp.to_clipboard()

In [25]:

def title_case(var):
    return ''.join(x for x in var.title() if not x.isspace())


def get_model_product_visits(file_path):
    df_ = visits_product(file_path,file_path1)
    df_['Date'] = pd.to_datetime(df_['Date'])
    df_result = pd.DataFrame(columns=['Date'])
    for ven in df_['Product'].unique():
        temp = df_[df_['Product']==ven]
        var = title_case(temp['Platform'].values[0])+"_"+ven
        df_grp = temp.groupby(['Date', 'Platform', 'Product']).sum().reset_index()
        df_grp = df_grp.drop(['Platform', 'Product'], axis=1)
        df_grp = df_grp.rename({'Value':var+"_Visits", 'Goal':var+"_Goal"}, axis=1)
        df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
    df_grp = visits_product_aggregate(file_path)
    df_grp = df_grp.drop('% of Goal', axis=1)
    df_grp = df_grp.rename({'Value':"Overall_Product_Visits", 'Goal':"Overall_Product_Goal"}, axis=1)
    df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
    df_result['US Weather Impact Rating'] = df_result['US Weather Impact Rating'].astype('float')
    return df_result


In [26]:
# Call the function
file_path = "C:\\Users\\DeepakArun\\Desktop\\TWC\\Data\\"
file_path1 = "C:\\Users\\DeepakArun\\Downloads\\new data\\"  # Replace with your actual file path
df_result = get_model_product_visits(file_path)

TypeError: float() argument must be a string or a real number, not 'NAType'

In [29]:
def get_organic_search_google(file_path):
    df_v1 = pd.read_excel(file_path+"Search Impressions - Google Only - 6.18.22-10.17.23.xlsx", sheet_name='Dates')
    df_v3 = pd.read_excel(file_path+"Organic Search Impressions - Google Only - 1.1.24-5.31.24 (1).xlsx", sheet_name='Dates')
    df_v2 = pd.read_excel(file_path+"Search Impressions - Google Only - 10.18.23-12.31.23 FIXED.xlsx", sheet_name='Dates')
    df_ = pd.concat([df_v1, df_v2])
    df_ = pd.concat([df_, df_v3])
    df_['Date'] = pd.to_datetime(df_['Date'])
    return df_

In [31]:
# Call the function
file_path = "C:\\Users\\DeepakArun\\Desktop\\TWC\\Data\\"
df_organic= get_organic_search_google(file_path)

In [56]:
df_organic.to_clipboard()

## User Acquisition - Media Spend, Impression and Clicks

In [36]:
def mkt_media_spend_pre_processing(df_, paid_media_sheet):
    df_columns = list(df_.columns)
    if df_['Spend'].dtypes == 'object':
        df_['Spend'] = df_['Spend'].str.replace(',', '').str.replace('$', '').astype('float')
    if 'clicks' in df_.columns:
            df_=df_.rename({'clicks':'Clicks'},axis=1)
    if 'Clicks' in df_.columns and df_['Clicks'].dtypes == 'object':
        df_['Clicks'] = df_['Clicks'].replace({'\$': ''}, regex=True).replace({'\,': ''}, regex=True)
        df_['Clicks'] = df_['Clicks'].astype('float')
    if paid_media_sheet == 'IronSource Sonic':
        df_=df_.rename({'OS':'event_date', 'Day':'OS'}, axis=1)
    if paid_media_sheet == 'Persona.ly':
        df_=df_[~df_['Platform'].isna()]
    if 'event_date' in list(df_.columns):
        df_=df_.rename({'event_date':'Day'}, axis=1)
    if paid_media_sheet == 'Persona.ly':
        df_['new_date'] = df_['Day']
        df_['format'] = 1
        df_.loc[df_.Day.str.contains('/')==True, 'format'] = 2
        df_.loc[df_.format == 2, 'new_date'] = pd.to_datetime(df_.loc[df_.format == 2, 'Day'], format = '%m/%d/%y').dt.strftime('%Y-%m-%d')
        df_.loc[df_.Date_Issue_flag == 1, 'new_date'] = pd.to_datetime(df_.loc[df_.Date_Issue_flag == 1, 'Day'], format = '%Y-%d-%m %H:%M:%S').dt.strftime('%Y-%m-%d')
        df_.loc[df_.format == 1, 'new_date'] = pd.to_datetime(df_.loc[df_.format == 1, 'Day'], format = '%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
        df_.loc[df_.Date_Issue_flag == 1, 'new_date'] = pd.to_datetime(df_.loc[df_.Date_Issue_flag == 1, 'Day'], format = '%Y-%d-%m %H:%M:%S').dt.strftime('%Y-%m-%d')
        df_ = df_.drop('Date_Issue_flag', axis=1)
    else:
        df_['format'] = 1
        df_.loc[df_.Day.str.contains('/')==True, 'format'] = 2
        df_.loc[df_.format == 1, 'new_date'] = pd.to_datetime(df_.loc[df_.format == 1, 'Day'], format = '%Y-%d-%m %H:%M:%S').dt.strftime('%Y-%m-%d')
        if paid_media_sheet == 'Bidease':
            pattern = r'\b\d{2}/\d{2}/\d{2}\b'
            df_.loc[((df_.format == 2) & (df_.Day.str.contains(pattern, regex=True))), 'format'] = 3
            df_.loc[df_.format == 2, 'new_date'] = pd.to_datetime(df_.loc[df_.format == 2, 'Day'], format = '%m/%d/%Y').dt.strftime('%Y-%m-%d')
            df_.loc[df_.format == 3, 'new_date'] = pd.to_datetime(df_.loc[df_.format == 3, 'Day'], format = '%m/%d/%y').dt.strftime('%Y-%m-%d')
        else:
            df_.loc[df_.format == 2, 'new_date'] = pd.to_datetime(df_.loc[df_.format == 2, 'Day'], format = '%m/%d/%y').dt.strftime('%Y-%m-%d')
    if 'event_date' in df_columns:
        df_=df_.rename({'Day':'event_date'}, axis=1)
        date_var = 'event_date'
    else:
        date_var = 'Day'
    df_[date_var] = df_['new_date']
    df_ = df_.drop(['format', 'new_date'], axis=1)
    return df_, date_var

In [34]:
# Call the function
file_path = "C:\\Users\\DeepakArun\\Desktop\\TWC\\Data\\"
df_pricing_android= pricing_android(file_path)

In [35]:
df_pricing_android

,Media_source,Date,Installs,Unnamed: 3,a,2021-09-01 00:00:00,Unnamed: 6,Unnamed: 7
0,appia_int,2021-09-01,65.0,NaN,appia_int,147.0,NaN,82.0
1,aura_int,2021-09-01,745.0,NaN,aura_int,747.0,NaN,2.0
2,bidease_int,2021-09-01,101.0,NaN,bidease_int,102.0,NaN,1.0
3,crossinstall_int,2021-09-01,35.0,NaN,crossinstall_int,35.0,NaN,0.0
4,digitalturbine_int,2021-09-01,958.0,NaN,digitalturbine_int,963.0,NaN,5.0
...,...,...,...,...,...,...,...,...
18603,EditorialReferralPM,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN
18604,aura_int,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN
18605,House_Backfill_INTL,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN
18606,None,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN


## Brand - Media Spend, Impression (US Brand Basis)

In [52]:

def US_brand_media_spend_impression(file_path):
    df_v1 = pd.read_excel(file_path+"US Brand - Basis - 2021-2023.xlsx")
    df_v2 = pd.read_excel(file_path+"US Brand - Basis - 10.16.23-12.31.23.xlsx")
    df_v3 = pd.read_excel(file_path+"TWC_MMM_Brand_Data File_01.01.24-05.31.24.xlsx")
    df_ = pd.concat([df_v1, df_v2])
    df_ = pd.concat([df_, df_v3]) 
    df_['Date'] = pd.to_datetime(df_['Date'])
    return df_
    
def get_model_brand_media_spend_impression(file_path):
    df_ = US_brand_media_spend_impression(file_path)
    df_result= pd.DataFrame(columns=['Date'])
    # for ven in df_['Vendor'].unique():
    #     temp = df_[df_['Vendor']==ven]
    #     var = temp['Advertising Type'].values[0]+"_"+ven
    #     df_grp = temp.groupby(['Date', 'Vendor', 'Advertising Type']).sum().reset_index()
    #     df_grp = df_grp.drop(['Vendor', 'Advertising Type'], axis = 1)
    #     df_grp = df_grp.rename({'Impressions':var+"_Impressions", 'Spend':var+"_Spend"}, axis=1)
    #     df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
    # for ven in df_['Advertising Type'].unique():
    #     temp = df_[df_['Advertising Type']==ven]
    #     var = temp['Advertising Type'].values[0]
    #     df_grp = temp.groupby(['Date', 'Advertising Type']).sum().reset_index()
    #     df_grp = df_grp.drop(['Advertising Type'], axis = 1)
    #     df_grp = df_grp.rename({'Impressions':var+"_Impressions", 'Spend':var+"_Spend"}, axis=1)
    #     df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
    df_grp = df_.groupby(['Date']).sum().reset_index()
    df_grp = df_grp.rename({'Impressions':"Brand_Impressions", 'Spend':"Brand_Spend"}, axis=1)
    df_result = pd.merge(df_result, df_grp, on='Date', how='outer')
    return df_result

In [53]:
file_path = "C:\\Users\\DeepakArun\\Desktop\\TWC\\Data\\"
result_df = get_model_brand_media_spend_impression(file_path)

In [55]:
result_df.to_clipboard()

In [57]:
# SEO Clicks - For Organic Search Clicks Imputation for missing data
def seo_pre_processing(df_21, df_22, df_23):
    df_21.columns = df_21.columns.str.strip()
    df_21['Segment'] = df_21['Segment'].str.strip()
    df_22.columns = df_22.columns.str.strip()
    df_22['Segment'] = df_22['Segment'].str.strip()
    df_23.columns = df_23.columns.str.strip()
    df_23['Segment'] = df_23['Segment'].str.strip()
    df_ = pd.merge(df_21, df_22, on='Segment', how='outer')
    df_ = pd.merge(df_, df_23, on='Segment', how='outer')
    df_columns = df_.columns
    df_columns_to_include = df_columns[df_columns>'2021-08-01']
    df_ = df_[df_columns_to_include]
    df_.set_index('Segment', inplace=True)
    df_ = df_.T
    df_=df_.reset_index()
    df_=df_.rename({'index':'Date'}, axis=1)
    return df_

def seo_get_combined_column(df_):
    df_['Desktop & Mobile SEO Clicks (Combined)'] = df_['Organic Search; United States; desktop'] + df_['Organic Search; United States; mobile web']
    df_ = df_[['Date', 'Desktop & Mobile SEO Clicks (Combined)']]
    return df_

def seo_clicks(file_path):
    df_21 = pd.read_excel(file_path+"US Web SEO Visits - 2021.xlsx", skiprows=6)
    df_22 = pd.read_excel(file_path+"US Web SEO Visits - 2022.xlsx", skiprows=6)
    df_23 = pd.read_excel(file_path+"US Web SEO Visits - 2023 1.1-10.16.xlsx", skiprows=6)
    df_ = seo_pre_processing(df_21, df_22, df_23)
    df_ = seo_get_combined_column(df_)
    return df_

In [58]:
file_path = "C:\\Users\\DeepakArun\\Desktop\\TWC\\Data\\"
seo_clicks_data = seo_clicks(file_path)

In [59]:
seo_clicks_data

Segment,Date,Desktop & Mobile SEO Clicks (Combined)
0,2021-08-02,613985
1,2021-08-03,1188368
2,2021-08-04,1168879
3,2021-08-05,1191761
4,2021-08-06,1244922
...,...,...
801,2023-10-12,1888616
802,2023-10-13,1755470
803,2023-10-14,1701830
804,2023-10-15,1986373


In [60]:
df = pd.read_excel("C:\\Users\\DeepakArun\\Desktop\\social media.xlsx")

In [61]:
df['PublishedTime'] = pd.to_datetime(df['PublishedTime']).dt.date


In [64]:
df

,PublishedTime,Channel,Total Engagements (SUM),Estimated Clicks (SUM),Post Reach (SUM)
0,2024-04-27,TIKTOK_BUSINESS,1497509,0,19040277
1,2024-04-29,INSTAGRAM,854811,0,13122822
2,2024-04-28,TIKTOK_BUSINESS,545779,0,7519695
3,2024-05-06,TIKTOK_BUSINESS,402526,0,6138803
4,2024-01-16,TIKTOK_BUSINESS,317752,0,4112836
...,...,...,...,...,...
9871,2024-04-26,INSTAGRAM,0,0,4505
9872,2024-04-26,FBPAGE,0,0,1
9873,2024-04-26,FBPAGE,0,0,1
9874,2024-04-26,FBPAGE,0,0,1


In [65]:
grouped_df = df.groupby(['PublishedTime', 'Channel']).agg({
    'Total Engagements (SUM)': 'sum',
    'Estimated Clicks (SUM)': 'sum',
    'Post Reach (SUM)': 'sum'
}).reset_index()


In [67]:
grouped_df.to_clipboard()

In [70]:
import pandas as pd

# Create the first dataset (simulated)
data1 = {
    'Date': ['01-09-2021', '02-09-2021', '03-09-2021', '04-09-2021', '05-09-2021', '06-09-2021', '07-09-2021', '08-09-2021', '09-09-2021', '10-09-2021', '11-09-2021', '12-09-2021'],
    'SocialEng_Facebook_Impressions': [4805113, 4013773, 2120546, 1239395, 184223, 591541, 2843076, 2412898, 2044945, 1507319, 829921, 894616],
    'SocialEng_Facebook_Total_Engagements': [31079, 29663, 6469, 4812, 304, 1810, 10243, 9623, 7739, 5262, 4229, 3556],
    'SocialEng_Facebook_Estimated_Clicks': [20632, 87579, 12172, 18746, 107, 18672, 17190, 28964, 11907, 19230, 4099, 11091],
    'SocialEng_LinkedIn_Impressions': [107826, 88570, 64859, 229, 189, 7182, 59593, 113147, 183006, 454600, 0, 8434],
    'SocialEng_LinkedIn_Total_Engagements': [1804, 1525, 1317, 31, 8, 97, 3083, 1853, 3467, 5809, 0, 366],
    'SocialEng_LinkedIn_Estimated_Clicks': [1258, 1681, 258, 33, 9, 353, 326, 2123, 7084, 1871, 0, 145],
    'SocialEng_YouTube_Impressions': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'SocialEng_YouTube_Estimated_Clicks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'SocialEng_Instagram_Impressions': [345169, 532110, 265977, 113167, 0, 343892, 488712, 488745, 397377, 434703, 217868, 255943],
    'SocialEng_Instagram_Total_Engagements': [9429, 18561, 9422, 1773, 0, 9364, 20156, 14653, 12815, 16214, 5152, 7822],
    'SocialEng_Instagram_Estimated_Clicks': [0, 0, 0, 0, 0, 0, 0, 16, 0, 4, 0, 0],
    'SocialEng_Twitter_Impressions': [63111321, 41235963, 22267851, 15871599, 16173193, 16819947, 60658580, 49934875, 42299904, 56995436, 24179834, 26497614],
    'SocialEng_Twitter_Total_Engagements': [6126, 5438, 1711, 564, 896, 1067, 4606, 5166, 3063, 1799, 1778, 1599],
    'SocialEng_Twitter_Estimated_Clicks': [745, 657, 468, 53, 46, 148, 708, 1868, 789, 351, 65, 62],
    'SocialEng_TikTok_Impressions': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'SocialEng_TikTok_Total_Engagements': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}

df1 = pd.DataFrame(data1)

# Create the second dataset (simulated)
data2 = {
    'PublishedTime': ['01-01-2024', '01-01-2024', '01-01-2024', '01-01-2024', '01-01-2024', '01-01-2024', '02-01-2024', '02-01-2024', '02-01-2024', '02-01-2024', '02-01-2024', '03-01-2024', '03-01-2024', '03-01-2024'],
    'Channel': ['FBPAGE', 'INSTAGRAM', 'LINKEDIN_COMPANY', 'TIKTOK_BUSINESS', 'TWITTER', 'YOUTUBE', 'FBPAGE', 'INSTAGRAM', 'TIKTOK_BUSINESS', 'TWITTER', 'YOUTUBE', 'FBPAGE', 'INSTAGRAM', 'LINKEDIN_COMPANY'],
    'Total Engagements (SUM)': [745, 5491, 75, 1349, 448, 480, 533, 30288, 11453, 1124, 1331, 521, 13383, 33],
    'Estimated Clicks (SUM)': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Post Reach (SUM)': [226380, 187433, 4297, 28093, 18650468, 0, 191230, 839858, 271525, 27975702, 0, 122724, 435379, 1852],
}

df2 = pd.DataFrame(data2)

# Normalize column names for df2 to match df1
# Create a new DataFrame to hold the combined data
df_combined = df1.copy()

# Map df2 data to df1's columns
for index, row in df2.iterrows():
    date = row['PublishedTime']
    channel = row['Channel']
    engagements = row['Total Engagements (SUM)']
    estimated_clicks = row['Estimated Clicks (SUM)']
    post_reach = row['Post Reach (SUM)']
    
    if channel == 'FBPAGE':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': post_reach,
            'SocialEng_Facebook_Total_Engagements': engagements,
            'SocialEng_Facebook_Estimated_Clicks': estimated_clicks,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'INSTAGRAM':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': post_reach,
            'SocialEng_Instagram_Total_Engagements': engagements,
            'SocialEng_Instagram_Estimated_Clicks': estimated_clicks,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'LINKEDIN_COMPANY':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': post_reach,
            'SocialEng_LinkedIn_Total_Engagements': engagements,
            'SocialEng_LinkedIn_Estimated_Clicks': estimated_clicks,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'TIKTOK_BUSINESS':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': post_reach,
            'SocialEng_TikTok_Total_Engagements': engagements,
        }, ignore_index=True)
    elif channel == 'TWITTER':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': post_reach,
            'SocialEng_Twitter_Total_Engagements': engagements,
            'SocialEng_Twitter_Estimated_Clicks': estimated_clicks,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'YOUTUBE':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': post_reach,
            'SocialEng_YouTube_Estimated_Clicks': estimated_clicks,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)

# Print the combined DataFrame
df_combined.tail()


,Date,SocialEng_Facebook_Impressions,SocialEng_Facebook_Total_Engagements,SocialEng_Facebook_Estimated_Clicks,SocialEng_LinkedIn_Impressions,SocialEng_LinkedIn_Total_Engagements,SocialEng_LinkedIn_Estimated_Clicks,SocialEng_YouTube_Impressions,SocialEng_YouTube_Estimated_Clicks,SocialEng_Instagram_Impressions,SocialEng_Instagram_Total_Engagements,SocialEng_Instagram_Estimated_Clicks,SocialEng_Twitter_Impressions,SocialEng_Twitter_Total_Engagements,SocialEng_Twitter_Estimated_Clicks,SocialEng_TikTok_Impressions,SocialEng_TikTok_Total_Engagements
21,02-01-2024,0,0,0,0,0,0,0,0,0,0,0,27975702,1124,0,0,0
22,02-01-2024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,03-01-2024,122724,521,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,03-01-2024,0,0,0,0,0,0,0,0,435379,13383,0,0,0,0,0,0
25,03-01-2024,0,0,0,1852,33,0,0,0,0,0,0,0,0,0,0,0


In [73]:
import pandas as pd

# Create the first dataset (simulated)
data1 = {
    'Date': ['01-09-2021', '02-09-2021', '03-09-2021', '04-09-2021', '05-09-2021', '06-09-2021', '07-09-2021', '08-09-2021', '09-09-2021', '10-09-2021', '11-09-2021', '12-09-2021'],
    'SocialEng_Facebook_Impressions': [4805113, 4013773, 2120546, 1239395, 184223, 591541, 2843076, 2412898, 2044945, 1507319, 829921, 894616],
    'SocialEng_Facebook_Total_Engagements': [31079, 29663, 6469, 4812, 304, 1810, 10243, 9623, 7739, 5262, 4229, 3556],
    'SocialEng_Facebook_Estimated_Clicks': [20632, 87579, 12172, 18746, 107, 18672, 17190, 28964, 11907, 19230, 4099, 11091],
    'SocialEng_LinkedIn_Impressions': [107826, 88570, 64859, 229, 189, 7182, 59593, 113147, 183006, 454600, 0, 8434],
    'SocialEng_LinkedIn_Total_Engagements': [1804, 1525, 1317, 31, 8, 97, 3083, 1853, 3467, 5809, 0, 366],
    'SocialEng_LinkedIn_Estimated_Clicks': [1258, 1681, 258, 33, 9, 353, 326, 2123, 7084, 1871, 0, 145],
    'SocialEng_YouTube_Impressions': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'SocialEng_YouTube_Estimated_Clicks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'SocialEng_Instagram_Impressions': [345169, 532110, 265977, 113167, 0, 343892, 488712, 488745, 397377, 434703, 217868, 255943],
    'SocialEng_Instagram_Total_Engagements': [9429, 18561, 9422, 1773, 0, 9364, 20156, 14653, 12815, 16214, 5152, 7822],
    'SocialEng_Instagram_Estimated_Clicks': [0, 0, 0, 0, 0, 0, 0, 16, 0, 4, 0, 0],
    'SocialEng_Twitter_Impressions': [63111321, 41235963, 22267851, 15871599, 16173193, 16819947, 60658580, 49934875, 42299904, 56995436, 24179834, 26497614],
    'SocialEng_Twitter_Total_Engagements': [6126, 5438, 1711, 564, 896, 1067, 4606, 5166, 3063, 1799, 1778, 1599],
    'SocialEng_Twitter_Estimated_Clicks': [745, 657, 468, 53, 46, 148, 708, 1868, 789, 351, 65, 62],
    'SocialEng_TikTok_Impressions': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'SocialEng_TikTok_Total_Engagements': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}

df1 = pd.DataFrame(data1)

# Create the second dataset (simulated)
data2 = {
    'PublishedTime': ['01-01-2024', '01-01-2024', '01-01-2024', '01-01-2024', '01-01-2024', '01-01-2024', '02-01-2024', '02-01-2024', '02-01-2024', '02-01-2024', '02-01-2024', '03-01-2024', '03-01-2024', '03-01-2024'],
    'Channel': ['FBPAGE', 'INSTAGRAM', 'LINKEDIN_COMPANY', 'TIKTOK_BUSINESS', 'TWITTER', 'YOUTUBE', 'FBPAGE', 'INSTAGRAM', 'TIKTOK_BUSINESS', 'TWITTER', 'YOUTUBE', 'FBPAGE', 'INSTAGRAM', 'LINKEDIN_COMPANY'],
    'Total Engagements (SUM)': [745, 5491, 75, 1349, 448, 480, 533, 30288, 11453, 1124, 1331, 521, 13383, 33],
    'Estimated Clicks (SUM)': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Post Reach (SUM)': [226380, 187433, 4297, 28093, 18650468, 0, 191230, 839858, 271525, 27975702, 0, 122724, 435379, 1852],
}
df2 = pd.DataFrame(data2)
# Extract date from datetime in df1

# Rename columns for consistency
df2.rename(columns={
    'PublishedTime': 'Date',
    'Total Engagements (SUM)': 'Total Engagements',
    'Estimated Clicks (SUM)': 'Estimated Clicks',
    'Post Reach (SUM)': 'Post Reach'
}, inplace=True)
df2['Date'] = pd.to_datetime(df2['Date']).dt.date

# Initialize combined DataFrame with necessary columns
columns = [
    'Date',
    'SocialEng_Facebook_Impressions',
    'SocialEng_Facebook_Total_Engagements',
    'SocialEng_Facebook_Estimated_Clicks',
    'SocialEng_LinkedIn_Impressions',
    'SocialEng_LinkedIn_Total_Engagements',
    'SocialEng_LinkedIn_Estimated_Clicks',
    'SocialEng_YouTube_Impressions',
    'SocialEng_YouTube_Estimated_Clicks',
    'SocialEng_Instagram_Impressions',
    'SocialEng_Instagram_Total_Engagements',
    'SocialEng_Instagram_Estimated_Clicks',
    'SocialEng_Twitter_Impressions',
    'SocialEng_Twitter_Total_Engagements',
    'SocialEng_Twitter_Estimated_Clicks',
    'SocialEng_TikTok_Impressions',
    'SocialEng_TikTok_Total_Engagements'
]
df_combined = pd.DataFrame(columns=columns)

# Map df2 data to df_combined's columns
for index, row in df2.iterrows():
    date = row['Date']
    channel = row['Channel']
    engagements = row['Total Engagements']
    estimated_clicks = row['Estimated Clicks']
    post_reach = row['Post Reach']
    
    if channel == 'FBPAGE':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': post_reach,
            'SocialEng_Facebook_Total_Engagements': engagements,
            'SocialEng_Facebook_Estimated_Clicks': estimated_clicks,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'INSTAGRAM':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': post_reach,
            'SocialEng_Instagram_Total_Engagements': engagements,
            'SocialEng_Instagram_Estimated_Clicks': estimated_clicks,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'LINKEDIN_COMPANY':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': post_reach,
            'SocialEng_LinkedIn_Total_Engagements': engagements,
            'SocialEng_LinkedIn_Estimated_Clicks': estimated_clicks,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'TIKTOK_BUSINESS':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': post_reach,
            'SocialEng_TikTok_Total_Engagements': engagements,
        }, ignore_index=True)
    elif channel == 'TWITTER':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': post_reach,
            'SocialEng_Twitter_Total_Engagements': engagements,
            'SocialEng_Twitter_Estimated_Clicks': estimated_clicks,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'YOUTUBE':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': post_reach,
            'SocialEng_YouTube_Estimated_Clicks': estimated_clicks,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)

# Combine the original df1 and the mapped df2
combined_df = pd.concat([df1, df_combined], ignore_index=True)

# Group by date and sum values
final_df = combined_df.groupby('Date').sum().reset_index()

# Save the final DataFrame to CSV
#final_df.to_csv('/mnt/data/final_combined_data.csv', index=False)

final_df.tail()


,Date,SocialEng_Facebook_Impressions,SocialEng_Facebook_Total_Engagements,SocialEng_Facebook_Estimated_Clicks,SocialEng_LinkedIn_Impressions,SocialEng_LinkedIn_Total_Engagements,SocialEng_LinkedIn_Estimated_Clicks,SocialEng_YouTube_Impressions,SocialEng_YouTube_Estimated_Clicks,SocialEng_Instagram_Impressions,SocialEng_Instagram_Total_Engagements,SocialEng_Instagram_Estimated_Clicks,SocialEng_Twitter_Impressions,SocialEng_Twitter_Total_Engagements,SocialEng_Twitter_Estimated_Clicks,SocialEng_TikTok_Impressions,SocialEng_TikTok_Total_Engagements
10,08-09-2021,2412898,9623,28964,113147,1853,2123,0,0,488745,14653,16,49934875,5166,1868,0,0
11,09-09-2021,2044945,7739,11907,183006,3467,7084,0,0,397377,12815,0,42299904,3063,789,0,0
12,10-09-2021,1507319,5262,19230,454600,5809,1871,0,0,434703,16214,4,56995436,1799,351,0,0
13,11-09-2021,829921,4229,4099,0,0,0,0,0,217868,5152,0,24179834,1778,65,0,0
14,12-09-2021,894616,3556,11091,8434,366,145,0,0,255943,7822,0,26497614,1599,62,0,0


In [77]:
import pandas as pd

df1 = pd.read_excel("C:\\Users\\DeepakArun\\Desktop\\old social media.xlsx")


df2 = pd.read_excel("C:\\Users\\DeepakArun\\Desktop\\new social media.xlsx")
# Extract date from datetime in df1

# Rename columns for consistency
df2.rename(columns={
    'PublishedTime': 'Date',
    'Total Engagements (SUM)': 'Total Engagements',
    'Estimated Clicks (SUM)': 'Estimated Clicks',
    'Post Reach (SUM)': 'Post Reach'
}, inplace=True)
df2['Date'] = pd.to_datetime(df2['Date']).dt.date

# Initialize combined DataFrame with necessary columns
columns = [
    'Date',
    'SocialEng_Facebook_Impressions',
    'SocialEng_Facebook_Total_Engagements',
    'SocialEng_Facebook_Estimated_Clicks',
    'SocialEng_LinkedIn_Impressions',
    'SocialEng_LinkedIn_Total_Engagements',
    'SocialEng_LinkedIn_Estimated_Clicks',
    'SocialEng_YouTube_Impressions',
    'SocialEng_YouTube_Estimated_Clicks',
    'SocialEng_Instagram_Impressions',
    'SocialEng_Instagram_Total_Engagements',
    'SocialEng_Instagram_Estimated_Clicks',
    'SocialEng_Twitter_Impressions',
    'SocialEng_Twitter_Total_Engagements',
    'SocialEng_Twitter_Estimated_Clicks',
    'SocialEng_TikTok_Impressions',
    'SocialEng_TikTok_Total_Engagements'
]
df_combined = pd.DataFrame(columns=columns)

# Map df2 data to df_combined's columns
for index, row in df2.iterrows():
    date = row['Date']
    channel = row['Channel']
    engagements = row['Total Engagements']
    estimated_clicks = row['Estimated Clicks']
    post_reach = row['Post Reach']
    
    if channel == 'FBPAGE':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': post_reach,
            'SocialEng_Facebook_Total_Engagements': engagements,
            'SocialEng_Facebook_Estimated_Clicks': estimated_clicks,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'INSTAGRAM':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': post_reach,
            'SocialEng_Instagram_Total_Engagements': engagements,
            'SocialEng_Instagram_Estimated_Clicks': estimated_clicks,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'LINKEDIN_COMPANY':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': post_reach,
            'SocialEng_LinkedIn_Total_Engagements': engagements,
            'SocialEng_LinkedIn_Estimated_Clicks': estimated_clicks,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'TIKTOK_BUSINESS':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': post_reach,
            'SocialEng_TikTok_Total_Engagements': engagements,
        }, ignore_index=True)
    elif channel == 'TWITTER':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': 0,
            'SocialEng_YouTube_Estimated_Clicks': 0,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': post_reach,
            'SocialEng_Twitter_Total_Engagements': engagements,
            'SocialEng_Twitter_Estimated_Clicks': estimated_clicks,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)
    elif channel == 'YOUTUBE':
        df_combined = df_combined.append({
            'Date': date,
            'SocialEng_Facebook_Impressions': 0,
            'SocialEng_Facebook_Total_Engagements': 0,
            'SocialEng_Facebook_Estimated_Clicks': 0,
            'SocialEng_LinkedIn_Impressions': 0,
            'SocialEng_LinkedIn_Total_Engagements': 0,
            'SocialEng_LinkedIn_Estimated_Clicks': 0,
            'SocialEng_YouTube_Impressions': post_reach,
            'SocialEng_YouTube_Estimated_Clicks': estimated_clicks,
            'SocialEng_Instagram_Impressions': 0,
            'SocialEng_Instagram_Total_Engagements': 0,
            'SocialEng_Instagram_Estimated_Clicks': 0,
            'SocialEng_Twitter_Impressions': 0,
            'SocialEng_Twitter_Total_Engagements': 0,
            'SocialEng_Twitter_Estimated_Clicks': 0,
            'SocialEng_TikTok_Impressions': 0,
            'SocialEng_TikTok_Total_Engagements': 0,
        }, ignore_index=True)

# Combine the original df1 and the mapped df2
combined_df = pd.concat([df1, df_combined], ignore_index=True)

# Group by date and sum values
final_df = combined_df.groupby('Date').sum().reset_index()

# Save the final DataFrame to CSV
#final_df.to_csv('/mnt/data/final_combined_data.csv', index=False)

final_df.tail()


,Date,SocialEng_Facebook_Impressions,SocialEng_Facebook_Total_Engagements,SocialEng_Facebook_Estimated_Clicks,SocialEng_LinkedIn_Impressions,SocialEng_LinkedIn_Total_Engagements,SocialEng_LinkedIn_Estimated_Clicks,SocialEng_YouTube_Impressions,SocialEng_YouTube_Estimated_Clicks,SocialEng_Instagram_Impressions,SocialEng_Instagram_Total_Engagements,SocialEng_Instagram_Estimated_Clicks,SocialEng_Twitter_Impressions,SocialEng_Twitter_Total_Engagements,SocialEng_Twitter_Estimated_Clicks,SocialEng_TikTok_Impressions,SocialEng_TikTok_Total_Engagements
999,2024-05-27,1449201,12973,0,0.0,0.0,0.0,0,0,273707,10027,0,59081588,1776,0,27281,1233
1000,2024-05-28,598819,7072,0,1352.0,44.0,0.0,0,0,575176,26888,0,73690505,1462,0,561395,25358
1001,2024-05-29,1187431,10665,0,1994.0,63.0,0.0,0,0,551652,25326,0,57982981,1114,0,137878,8060
1002,2024-05-30,1141994,13621,0,1329.0,38.0,0.0,0,0,656495,35389,0,63782522,754,0,149773,6880
1003,2024-05-31,1080636,9088,33,3485.0,122.0,45.0,0,0,264256,7310,0,82071246,1138,0,63923,2396


In [78]:
final_df.to_clipboard()